In [1]:
import os
import gc
import re
import sys
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

## 一.瓜子车源数据匹配

In [16]:
def process_price_bn(df):
    if df['price_bn'] == '暂无报价':
        return np.NAN
    temp = re.findall(r'万',df['price_bn'])
    if len(temp) > 1:
        text = df['price_bn'].split('~')[1]
    else:
        text = df['price_bn']
    text = text.replace('万','')
    return float(text)

def delete_str_useless(df, column_name):
    """
    删除没用的字符
    """    
    text = df[column_name]
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('-', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

# 判断是否有重复final_text
origin_train = pd.read_csv('../tmp/train/origin_train.csv')
origin_train = origin_train.loc[(origin_train['source_id'].notnull())&(origin_train['detail_name'].notnull())
                                &(origin_train['transfer_owner'] == 0), :].reset_index(drop=True)
origin_train['detail_name'] = origin_train.apply(delete_str_useless, args=('detail_name',), axis=1)
origin_train = origin_train.rename(columns={'detail_name':'title'})
train_final = pd.read_csv('../tmp/train/train_final.csv')
train_final = train_final.rename(columns={'final_text':'title'})
data = train_final.groupby(['title'])['brand_slug'].count().reset_index()
data.loc[(data['brand_slug'] > 1), :]

,title,brand_slug


In [17]:
# data = wait_predict.loc[(wait_predict['domain'].isin(['guazi.com'])), :]
data = origin_train.copy()
print('瓜子车源数据量:', len(data))
final = data.merge(train_final, how='left', on=['title'])
final = final.sort_values(by=['title'])
match = final.loc[(final['online_year'].notnull())&(final['city'].notnull()), :].reset_index(drop=True)
print('瓜子车源数据匹配量:',len(match), '匹配率:', len(match)/len(data))
match.info()
match.to_csv('/home/ml/ProgramProject/evaluation-predict/tmp/train/car_source_match.csv', index=False)

瓜子车源数据量: 146002
瓜子车源数据匹配量: 136431 匹配率: 0.9344461034780346
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136431 entries, 0 to 136430
Data columns (total 25 columns):
id                      136431 non-null int64
source_id               136431 non-null float64
title                   136431 non-null object
year                    136431 non-null float64
month                   136431 non-null float64
mile                    136431 non-null float64
city                    136431 non-null object
color                   0 non-null float64
car_application         0 non-null float64
price                   136431 non-null float64
domain                  136431 non-null object
labels                  136431 non-null object
transfer_owner          136431 non-null int64
annual_insurance        136431 non-null object
compulsory_insurance    136431 non-null object
business_insurance      136431 non-null object
create_time             136431 non-null object
brand_slug              136431 non-nu